In [3]:
###FEATURE EXTRACTION

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
#Getting the data
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

IMAGE_SIZE = (224,224)
BATCH_SIZE = 64

train_ds = train_gen.flow_from_directory(train_dir,
                                         batch_size=BATCH_SIZE,
                                         target_size=IMAGE_SIZE,
                                         class_mode="categorical")
test_ds = test_gen.flow_from_directory(test_dir,
                                      batch_size=BATCH_SIZE,
                                      target_size=IMAGE_SIZE,
                                      class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [5]:
#callbacks
#tensorboard
import datetime

def create_tensorboard_callback(dir_name, exp_name):
    log_dir = dir_name + "/" + exp_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print("Saving TensorBoard log files to", logdir)
    return tensorboard_callback

In [11]:
#Creating models using tensorflow hub
import tensorflow_hub as hub
from tensorflow.keras import layers

def create_model(model_url, num_classes=10):
    """
    Takes a tf hub url and creates a Sequential model with it
    """
    feature_extractor_layer = hub.KerasLayer(model_url, 
                                             trainable=False,
                                            input_shape=IMAGE_SIZE+(3,))
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation="softmax")
    ])
    
    return model

In [12]:
efficientnet_v2_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/classification/2"

mobilenet_v3_url = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5"

efficientnet_model = create_model(efficientnet_v2_url)

mobilenet_model = create_model(mobilenet_v3_url)

Metal device set to: Apple M1 Max


2023-09-06 19:38:19.889614: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-06 19:38:19.890006: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
mobilenet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])

In [20]:
history_mobilenet = mobilenet_model.fit(train_ds,
                                             validation_data=test_ds,
                                             epochs=20,
                                             steps_per_epoch=len(train_ds),
                                             validation_steps=len(test_ds))

Epoch 1/10


2023-09-07 17:58:48.423830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - ETA: 0s - loss: 2.0835 - accuracy: 0.2787

2023-09-07 17:58:52.366132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 12s 784ms/step - loss: 2.0835 - accuracy: 0.2787 - val_loss: 1.6482 - val_accuracy: 0.5448
Epoch 2/10
12/12 [==============================] - 6s 546ms/step - loss: 1.3984 - accuracy: 0.6640 - val_loss: 1.1604 - val_accuracy: 0.7184
Epoch 3/10
12/12 [==============================] - 6s 548ms/step - loss: 1.0068 - accuracy: 0.8067 - val_loss: 0.8953 - val_accuracy: 0.7900
Epoch 4/10
12/12 [==============================] - 6s 534ms/step - loss: 0.7840 - accuracy: 0.8440 - val_loss: 0.7553 - val_accuracy: 0.8148
Epoch 5/10
12/12 [==============================] - 6s 541ms/step - loss: 0.6562 - accuracy: 0.8627 - val_loss: 0.6744 - val_accuracy: 0.8284
Epoch 6/10
12/12 [==============================] - 6s 552ms/step - loss: 0.5656 - accuracy: 0.8867 - val_loss: 0.6236 - val_accuracy: 0.8388
Epoch 7/10
12/12 [==============================] - 6s 576ms/step - loss: 0.5023 - accuracy: 0.8987 - val_loss: 0.5870 - val_accuracy: 0.8424
Epoch 8/10
12/12

In [21]:
mobilenet_model.evaluate(test_ds)

40/40 [==============================] - 5s 115ms/step - loss: 0.5275 - accuracy: 0.8504


[0.52747642993927, 0.8503999710083008]

In [23]:
efficientnet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])
history_efficientnet = efficientnet_model.fit(train_ds,
                                             validation_data=test_ds,
                                             epochs=20,
                                             steps_per_epoch=len(train_ds),
                                             validation_steps=len(test_ds))

Epoch 1/20


2023-09-07 18:25:57.816710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - ETA: 0s - loss: 428.8989 - accuracy: 0.1733

2023-09-07 18:26:02.646068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 13s 1s/step - loss: 428.8989 - accuracy: 0.1733 - val_loss: 400.4788 - val_accuracy: 0.1904
Epoch 2/20
12/12 [==============================] - 7s 576ms/step - loss: 259.8909 - accuracy: 0.2080 - val_loss: 235.5818 - val_accuracy: 0.3180
Epoch 3/20
12/12 [==============================] - 6s 567ms/step - loss: 165.4703 - accuracy: 0.2733 - val_loss: 70.7217 - val_accuracy: 0.2624
Epoch 4/20
12/12 [==============================] - 6s 544ms/step - loss: 58.3142 - accuracy: 0.4280 - val_loss: 37.2986 - val_accuracy: 0.5712
Epoch 5/20
12/12 [==============================] - 7s 583ms/step - loss: 29.4950 - accuracy: 0.5347 - val_loss: 26.8800 - val_accuracy: 0.6756
Epoch 6/20
12/12 [==============================] - 7s 571ms/step - loss: 17.8022 - accuracy: 0.6307 - val_loss: 7.4949 - val_accuracy: 0.7192
Epoch 7/20
12/12 [==============================] - 6s 544ms/step - loss: 9.1637 - accuracy: 0.7227 - val_loss: 11.1579 - val_accuracy: 0.7496
Ep

In [24]:
efficientnet_model.evaluate(test_ds)

40/40 [==============================] - 5s 118ms/step - loss: 9.2386 - accuracy: 0.6796


[9.238638877868652, 0.6796000003814697]